In [ ]:
from sklearn import linear_model, svm
from sklearn.model_selection import train_test_split, cross_val_score
import os, sys, time
import pandas as pd, numpy as np
from sklearn.feature_selection import chi2

In [ ]:
# define models
ridgecv = linear_model.RidgeCV(alphas=[0.1,1,10])
lasso = linear_model.Lasso(alpha=0.1)
ridge = linear_model.Ridge(alpha=0.1, fit_intercept=False)
svr = svm.SVR()

In [3]:
path = r'/home/test/prsnl/airline'
os.chdir(path)
os.getcwd()

'/home/test/prsnl/airline'

In [4]:
def read_files():
    global data2, plane, airports, carriers
    
    data_2007 = pd.read_csv('2007.csv')    
    print len(data_2007)
    data_2008 = pd.read_csv('2008.csv')
    print len(data_2008)
    data2 = pd.concat([data_2007, data_2008],axis=0)
    data2.reset_index(inplace=True)
    plane = pd.read_csv('plane-data.csv')

    

In [57]:
read_files()

7453215
7009728


In [58]:
# just take first 1 million rows from 2007 sheet
data = data2.ix[:10000000].copy()

In [59]:
#sys.getsizeof(data) / 1000000
del data2

In [60]:
Y_all = data.ix[:,'ArrDelay'].copy()

In [61]:
# Select below few columns

lst = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest']
out = pd.DataFrame()
type(out)

def create_dummies():
    out = pd.DataFrame()
    for col in lst:
        cat = pd.get_dummies(data.ix[:,col])
        #print cat.info()
        out = pd.concat([out,cat], axis=1)
    return out
        
# These are categorical columns. Transform them to dummy variables
# a df named out contains all these dummy variables
out = create_dummies()
out.info()

MemoryError: 

In [ ]:
# create df named out2 for elapsed time & distance, which are already in numeric format
out2 = data.ix[:,['CRSElapsedTime','Distance']]

In [ ]:
# Subset expected departure time into six 4-hour buckets
deptime = data.ix[:,'CRSDepTime']
per_series = []
for ind, row in deptime.iteritems():
    #print row
    if row < 400:
        period = 1
    elif row < 800:
        period = 2
    elif row < 1200:
        period = 3
    elif row < 1600:
        period = 4
    elif row < 2000:
        period = 5
    else:
        period = 6
    
    per_series.append(period)
    

# out3 will contain departure time dummy variables
out3 = pd.get_dummies(pd.Series(per_series))
    

In [ ]:
#uniq_tail = data.ix[:,'TailNum'].drop_duplicates().dropna()
#uniq_tail

In [ ]:
# This will identify unique tail numbers from 2007/2008 data; get year from plane-data, and build a list of years 
# for corresponding tail numbers
uniq_tail = data.ix[:,'TailNum'].drop_duplicates().dropna()
uniq_tail.reset_index()
print len(uniq_tail)
year_list = []
for ind, tail in uniq_tail.iteritems():
    #print 'ind ', ind
    year = plane.ix[plane.ix[:,'tailnum'] == tail,'year']
    year.reset_index(inplace=True,drop=True)
    #year = pd.to_numeric(year_ser)
    

    try:
        if year.empty:
            #print 'year1', year
            decade = '00-00'
            year_list.append(decade)
            continue
    except:
        print 'year2 ', year
        
    

    if year.item() < 1960:
        decade = '50-60'
    elif year.item() < 1970:
        decade = '60-70'
    elif year.item() < 1980:
        decade = '70-80'
    elif year.item() < 1990:
        decade = '80-90'
    elif year.item() < 2000:
        decade = '90-00'
    else:
        decade = '00-10'

        
    year_list.append(decade)


In [ ]:
# Combine tail number with corresponding year into a dataframe
year_series = pd.Series(year_list)
year_series.reset_index(inplace=True,drop=True)
uniq_tail.reset_index(inplace=True,drop=True)
print len(year_series)
tail_year = pd.concat([uniq_tail,year_series],axis=1)
#print year_series

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0.1, stratify=Y)

In [ ]:
# Memory error... reduce the count of rows
# Y is target
limit = 800000
Y = Y_all[:limit+1].copy()

In [ ]:
# Find whether there are any NaNs in target variable - Arrival delay
# There were few such rows...
Y_np = np.array(Y)
print 'Are any of the values Null for Y? ' ,  np.any(np.isnan(np.array(Y)))
print 'Are values finite for Y? ', np.any(np.isfinite(np.array(Y)))
print 'count of nulls in 3 diff approaches ', Y.isnull().sum().sum() , np.sum(np.isnan(Y_np)), \
        pd.isnull(Y_np).sum()

In [ ]:
# Find index of rows that contain NaN in target variable
null_inds = np.where(np.isnan(Y_np))[0]

In [ ]:
print null_inds, len(null_inds)

In [ ]:
# Remove rows Y where NaN was present in target variable
Y.drop(Y.index[null_inds], inplace=True)
Y_np = np.array(Y)

In [ ]:
Y_np.shape

# This is where actual modeling happens

In [ ]:
# Modeling

# Concatenate all the feature data frames
new_temp_X = data.ix[:limit,['CarrierDelay','WeatherDelay', 'NASDelay', 'SecurityDelay']]
temp_X = pd.concat([new_temp_X, out3.ix[:limit,], out2.ix[:limit,], out.ix[:limit,]],axis=1)

print temp_X.shape
print 'Count of nulls ' , np.isnan(temp_X).sum().sum()

temp_X.drop(temp_X.index[null_inds], inplace=True)
print 'Count of nulls after processing..' , np.isnan(temp_X).sum().sum()
print temp_X.shape
#len(temp_X[temp_X[:] == 0.0])


In [62]:
# print all column names
#temp_X.columns.values
#temp_X['CRSElapsedTime'].isnull().any()

In [65]:
clf = ridge
#clf = lasso
#clf = svr

In [66]:
st = time.time()
temp_X_np = np.array(temp_X)
score = cross_val_score(clf, temp_X_np, Y_np, cv=5)
print 'duration: ', time.time() - st
print 'Score; average score ' , score, np.average(score)

duration:  49.137706995
Score; average score  [ 0.46801901  0.58914478  0.5708272   0.67176553  0.71454111] 0.602859525262


In [67]:
pred = lasso.fit(temp_X_np, Y_np)

In [68]:
# Find linear regression coefficient. Second value is for weather delay
print pred.intercept_
pred.coef_[1]

3.64735818671


1.0218866235281814

In [69]:
# Find pvalue for weather delay
scores, pvalues = chi2(temp_X_np, Y_np)

#pvalues[np.where(pvalues[:] > 0.05)]
pvalues[:1]

array([ 0.])

# Find Pearson Correlation between weather & arrival delay

In [70]:
from scipy.stats import pearsonr

In [71]:
weather_vect = data.ix[:,'WeatherDelay'].copy()
arr_vect = data.ix[:,'ArrDelay'].copy()

In [72]:
weather_vect.shape, arr_vect.shape

((10000001,), (10000001,))

In [78]:
print 'nulls in weather & arrival delay vectors? ' , weather_vect.isnull().any(), arr_vect.isnull().any()
null_inds = np.where(arr_vect.isnull())
null2_inds = np.where(weather_vect.isnull())
print len(null_inds[0]), len(null2_inds[0])

# drop rows that have null in arrival delay vector 
weather_vect.drop(weather_vect.index[null_inds], inplace=True)
arr_vect.drop(arr_vect.index[null_inds], inplace=True)
weather_vect.drop(weather_vect.index[null2_inds], inplace=True)
arr_vect.drop(arr_vect.index[null2_inds], inplace=True)
print 'nulls in weather & arrival delay vectors? ' , weather_vect.isnull().any(), arr_vect.isnull().any()

nulls in weather & arrival delay vectors?  True False
0 1859563
nulls in weather & arrival delay vectors?  False False


In [79]:
pearsonr(weather_vect, arr_vect)

(0.28242394418128486, 0.0)